# Web Scraping partidas das copas do mundo 

A Copa do Mundo de 2022 está chegando e que melhor maneira de aprender ciência de dados e Python do que resolver um projeto do mundo real?

Mas não podemos iniciar um projeto sem dados, então, neste guia, usaremos Python e Beautiful Soup para extrair dados de todas as copas do mundo disputadas até agora (1930–2018) e da Copa do Mundo de 2022.


Neste projeto, usaremos bs4 para extrair dados do site, lxml para analisar documentos HTML e requests para enviar solicitações ao site de destino.

Aqui está o comando que você precisa executar no terminal para instalar essas bibliotecas: 

- pip install bs4 
- pip install lxml 
- pip install request



# 1.  Importações

In [23]:
import requests

In [24]:
import pandas as pd
import numpy  as np

In [25]:
from bs4 import BeautifulSoup

# 2. Coletar dados  (Parte 1) 

Vamos coletar dados de todas as copas do mundo jogadas até agora, vamos começar coletando dados de uma copa do mundo — Brasil 2014. Na parte 2, usaremos o código escrito na parte 1 para extrair dados de todas as copas do mundo.


In [27]:
web      = 'https://en.wikipedia.org/wiki/2014_FIFA_World_Cup'
response = requests.get(web) 
content  = response.text 
soup     = BeautifulSoup(content, 'lxml' )

## Extraindo todas as partidas da Copa do Mundo
Agora é hora de extrair as partidas de futebol na web. Para isso, temos que identificar um padrão que nos permita extrair não apenas uma, mas todas as partidas da competição.

Para encontrar facilmente um padrão, primeiro temos que inspecionar o site clicando com o botão direito do mouse e selecionando “Inspecionar”. Depois disso, as ferramentas do desenvolvedor aparecerão. 

![](img/ws1.jpg)

Como você pode ver, cada partida disputada está dentro de uma linha representada pelo nó HTML destacado em azul acima.

![](img/ws-2.jpg)

Agora, para extrair todas as correspondências com nosso soup, temos que usar o método .find_all. Este método precisa de 2 entradas: o nome da tag e o nome da classe.

In [138]:
grupo    = soup.find_all( 'div' , class_= 'sports-table-notes' )
partidas = soup.find_all( 'div' , class_= 'footballbox' )

In [33]:
type(partidas)

bs4.element.ResultSet

## Extraindo os dados de pontuação de cada partida
Agora que temos todas as correspondências dentro de nossa lista, precisamos percorrê-la para extrair informações específicas.

Neste caso, vamos extrair o time da casa/visitante e os dados de pontuação. Então vamos armazená-los dentro de listas vazias, para depois podermos colocá-los em uma tabela.

Para obter os dados do time da casa, precisamos primeiro inspecioná-los, depois copiar o nome da tag e o nome da classe. O mesmo vale para o placar e time visitante.

![](img/ws-3.jpg)

In [ ]:
class="fleft"

In [134]:
group = []
home  = []
score = []
away  = []
date  = []
place = []

for partida in partidas:
    home.append(partida.find('th', class_='fhome').get_text())
    score.append(partida.find('th', class_='fscore').get_text())
    away.append(partida.find('th', class_='faway').get_text())   
    date.append(partida.find('div', class_='fleft').get_text())
    place.append(partida.find('div', class_='fright').get_text())

In [141]:
home[:10]

['Brazil\xa0',
 'Mexico\xa0',
 'Brazil\xa0',
 'Cameroon\xa0',
 'Cameroon\xa0',
 'Croatia\xa0',
 'Spain\xa0',
 'Chile\xa0',
 'Australia\xa0',
 'Spain\xa0']

## Armazenando nossos dados em um dataframe e salvar em um arquivo CSV
Criaremos um dataframe a partir das listas criadas acima, além disso, criaremos uma coluna chamada “year” que conterá o ano da copa do mundo (2014 para este caso específico).

In [142]:
score_home = []
score_away = []

for i, s in enumerate(score): 
    score_home.append(s[0]) 
    score_away.append(s[2])
    
dict_partidas = {'home'      : home, 
                 'score_home': score_home, 
                 'away'      : away, 
                 'score_away': score_away, 
                 'date'      : date, 
                 'place'     : place}


df_partidas   = pd.DataFrame(dict_partidas)

df_partidas['year'] = 2014

In [143]:
df_partidas.head()

,home,score_home,away,score_away,date,place,year
0,Brazil,3,Croatia,1,12 June 201417:00 BRT (UTC−3),"Arena de São Paulo, São PauloAttendance: 62,10...",2014
1,Mexico,1,Cameroon,0,13 June 201413:00 BRT (UTC−3),"Arena das Dunas, NatalAttendance: 39,216Refere...",2014
2,Brazil,0,Mexico,0,17 June 201416:00 BRT (UTC−3),"Estádio Castelão, FortalezaAttendance: 60,342R...",2014
3,Cameroon,0,Croatia,4,18 June 201418:00 AMT (UTC−4),"Arena da Amazônia, ManausAttendance: 39,982Ref...",2014
4,Cameroon,1,Brazil,4,23 June 201417:00 BRT (UTC−3),"Estádio Nacional Mané Garrincha, BrasíliaAtten...",2014


- Salvar o dadaset 

In [144]:
df_partidas.to_csv('Data/fifa_worldcup_historical_data.csv', index=False)

# 3. Coletando dados de TODAS as Copas do Mundo (Parte 2)

Agora que sabemos como extrair as informações de uma copa do mundo, é hora de extrair todas as copas, primeiro precisamos encontrar um padrão nos links.

Vamos dar uma olhada nos links das copas do mundo 2014, 2018 e 2022
- https://en.wikipedia.org/wiki/2014_FIFA_World_Cup 
- https://en.wikipedia.org/wiki/2018_FIFA_World_Cup 
- https://en.wikipedia.org/wiki/2022_FIFA_World_Cup

Podemos notar um padrão nos links, são idênticos, exceto no ano em que ocorreu uma copa do mundo, sendo assim, podemos reescrever a varável web para considerar este padrão:

In [87]:
year = 2022
web  = 'https://en.wikipedia.org/wiki/{}_FIFA_World_Cup'.format(year)
print(web)

https://en.wikipedia.org/wiki/2022_FIFA_World_Cup


Agora vamos pegar todo o códificação e colocar em uma função que recebe uma entrada, neste caso será o ano. 

In [132]:
def get_partidas(year_:int): 
    
    web = 'https://en.wikipedia.org/wiki/{}_FIFA_World_Cup'.format(year_)
    
    home  = []
    score = []
    away  = []
    date  = []
    place = []

    for partida in partidas:
        home.append(partida.find('th', class_='fhome').get_text())
        score.append(partida.find('th', class_='fscore').get_text())
        away.append(partida.find('th', class_='faway').get_text())   
        date.append(partida.find('div', class_='fleft').get_text())
        place.append(partida.find('div', class_='fright').get_text())

    
    score_home = []
    score_away = []

    for i, s in enumerate(score): 
        score_home.append(s[0]) 
        score_away.append(s[2])

    dict_partidas = {'home'      : home, 
                     'score_home': score_home, 
                     'away'      : away, 
                     'score_away': score_away, 
                     'date'      : date, 
                     'place'     : place}

    _ = pd.DataFrame(dict_partidas)
    _['year'] = year_
        
    return _

In [149]:
df_partidas_hist = pd.DataFrame()

for year in range(1930, 2024, 4): 
    _ = get_partidas(year)
    df_partidas_hist = pd.concat([df_partidas_hist, _], axis=0)

In [150]:
df_partidas_hist.head()

,home,score_home,away,score_away,date,place,year
0,Brazil,3,Croatia,1,12 June 201417:00 BRT (UTC−3),"Arena de São Paulo, São PauloAttendance: 62,10...",1930
1,Mexico,1,Cameroon,0,13 June 201413:00 BRT (UTC−3),"Arena das Dunas, NatalAttendance: 39,216Refere...",1930
2,Brazil,0,Mexico,0,17 June 201416:00 BRT (UTC−3),"Estádio Castelão, FortalezaAttendance: 60,342R...",1930
3,Cameroon,0,Croatia,4,18 June 201418:00 AMT (UTC−4),"Arena da Amazônia, ManausAttendance: 39,982Ref...",1930
4,Cameroon,1,Brazil,4,23 June 201417:00 BRT (UTC−3),"Estádio Nacional Mané Garrincha, BrasíliaAtten...",1930


In [151]:
df_partidas_hist.to_csv('Data/fifa_worldcup_historical_data_hist.csv', index=False)

In [152]:
df_partidas_hist.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1536 entries, 0 to 63
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   home        1536 non-null   object
 1   score_home  1536 non-null   object
 2   away        1536 non-null   object
 3   score_away  1536 non-null   object
 4   date        1536 non-null   object
 5   place       1536 non-null   object
 6   year        1536 non-null   int64 
dtypes: int64(1), object(6)
memory usage: 96.0+ KB


Pronto temos o primeiro extrador das informações. 